In [2]:
from datetime import timedelta, datetime
import rsa_headers as rh
import rsa_data_summary as rd
# import main
import config
import queries
import uuid
from pandasql import sqldf
from sqlalchemy.dialects.postgresql import insert

files = r"C:\Users\MB27{}5851\Desktop\Temp\rsa_traffic_counts\SMEC RSA Files_GP PRM Sites_Dec21toFeb22\{}337_2{}22{}131.RSA"
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

testfile = r"C:\Users\MB2705851\Desktop\Temp\SYNTELL-test.csv"

In [3]:
GET_HSWIM_HEADER_IDS = """SELECT DISTINCT header_id from trafc.electronic_count_data_type_10 t10 
    where not exists (select header_id from trafc.electronic_count_header_hswim h where h.header_id = t10.header_id)
    and 
    (
        exists (select header_id from trafc.traffic_e_type10_wheel_mass m where m.type10_id = t10.data_id)
        or
        exists (select header_id from trafc.traffic_e_type10_axle_group_mass m where m.type10_id = t10.data_id)
        or
        exists (select header_id from trafc.traffic_e_type10_axle_mass m where m.type10_id = t10.data_id)
    )
    ;"""

HEADER_COLUMNS_QRY = """select * from trafc.electronic_count_header limit 1"""

In [4]:
header_columns = list(pd.read_sql_query(HEADER_COLUMNS_QRY,config.ENGINE).columns)

In [5]:
header_id = '534c87d9-aceb-4ccb-851e-2e7f7b101969'

In [19]:
header = pd.read_sql_query(f"select * from trafc.electronic_count_header where header_id = '{header_id}';", config.ENGINE)

In [24]:
pri_scheme = header['type_10_vehicle_classification_scheme_primary'][0]
sec_scheme = header['type_10_vehicle_classification_scheme_secondary'][0]

In [75]:
def wim_stations_header_insert_qrys(header_id):
    SELECT_TYPE10_QRY = f"""SELECT * FROM trafc.electronic_count_data_type_10 t10
        -- left join traf_lu.vehicle_classes_scheme08 c on c.id = t10.vehicle_class_code_primary_scheme
        where t10.header_id = '{header_id}'
        """
    AXLE_SPACING_SELECT_QRY = f"""SELECT 
        t10.id,
        t10.header_id, 
        t10.start_datetime,
        t10.edit_code,
        t10.vehicle_class_code_primary_scheme, 
        t10.vehicle_class_code_secondary_scheme,
        t10.direction,
        t10.axle_count,
        axs.axle_spacing_number,
        axs.axle_spacing_cm
        FROM trafc.electronic_count_data_type_10 t10
        inner join trafc.traffic_e_type10_axle_spacing axs ON axs.type10_id = t10.data_id
        where t10.header_id = '{header_id}'
        """
    WHEEL_MASS_SELECT_QRY = f"""SELECT 
        t10.id,
        t10.header_id, 
        t10.start_datetime,
        t10.edit_code,
        t10.vehicle_class_code_primary_scheme, 
        t10.vehicle_class_code_secondary_scheme,
        t10.direction,
        t10.axle_count,
        wm.wheel_mass_number,
        wm.wheel_mass,
        vm.kg as vehicle_mass_limit_kg,
        sum(wm.wheel_mass*2) over(partition by t10.id) as gross_mass
        FROM trafc.electronic_count_data_type_10 t10
        inner join trafc.traffic_e_type10_wheel_mass wm ON wm.type10_id = t10.data_id
        inner join traf_lu.gross_vehicle_mass_limits vm on vm.number_of_axles = t10.axle_count
        where t10.header_id = '{header_id}'
        """
    return SELECT_TYPE10_QRY, AXLE_SPACING_SELECT_QRY, WHEEL_MASS_SELECT_QRY

In [76]:
SELECT_TYPE10_QRY, AXLE_SPACING_SELECT_QRY, WHEEL_MASS_SELECT_QRY = wim_stations_header_insert_qrys(header_id)

In [77]:
def wim_stations_header_insert_dfs(SELECT_TYPE10_QRY, AXLE_SPACING_SELECT_QRY, WHEEL_MASS_SELECT_QRY):
    df = pd.read_sql_query(SELECT_TYPE10_QRY,config.ENGINE)
    df = df.fillna(0)
    df2 = pd.read_sql_query(AXLE_SPACING_SELECT_QRY,config.ENGINE)
    df2 = df2.fillna(0)
    df3 = pd.read_sql_query(WHEEL_MASS_SELECT_QRY,config.ENGINE)
    df3 = df3.fillna(0)
    return df, df2, df3

In [78]:
df, df2, df3 = wim_stations_header_insert_dfs(SELECT_TYPE10_QRY, AXLE_SPACING_SELECT_QRY, WHEEL_MASS_SELECT_QRY)

In [10]:
GVML = pd.read_sql_query(queries.GET_GROSS_VEHICLE_MASS_LIMITS, config.ENGINE)
AXLL = pd.read_sql_query(queries.GET_AXLE_LOAD_LIMITS, config.ENGINE)

In [90]:
if df['vehicle_class_code_secondary_scheme'].unique() is None:
    scheme = len(df['vehicle_class_code_primary_scheme'].unique())
    col_name = 'vehicle_class_code_primary_scheme'
else: 
    scheme = len(df['vehicle_class_code_secondary_scheme'].unique())
    col_name = 'vehicle_class_code_secondary_scheme'

In [87]:
if scheme in [16,17]:
    scheme_tbl = pd.read_sql_table(table_name='vehicle_classes_scheme08',schema='traf_lu', con=config.ENGINE)
elif scheme in [4,5]:
    scheme_tbl = pd.read_sql_table(table_name='vehicle_classes_scheme05',schema='traf_lu', con=config.ENGINE)
elif scheme in [2,3]:
    scheme_tbl = pd.read_sql_table(table_name='vehicle_classes_scheme01',schema='traf_lu', con=config.ENGINE)

In [91]:
scheme_tbl = scheme_tbl.rename(columns={'id':col_name})

In [93]:
df = pd.merge(df,scheme_tbl, on=col_name, how='outer')
# df.head()

In [54]:
for header_id in list(header_ids['header_id'].astype(str)):
    SELECT_TYPE10_QRY, AXLE_SPACING_SELECT_QRY, WHEEL_MASS_SELECT_QRY = wim_stations_header_insert_qrys(header_id)
    df, df2, df3 = wim_stations_header_insert_dfs(SELECT_TYPE10_QRY, AXLE_SPACING_SELECT_QRY, WHEEL_MASS_SELECT_QRY)
    if df2 is None or df3 is None:
        pass
    else:
        insert_string = wim_stations_header_insert(header_id, df, df2, df3)
        with config.ENGINE.connect() as conn:
            conn.execute(insert_string)
            print('COMPLETE')

COMPLETE
COMPLETE
COMPLETE
COMPLETE


In [39]:
df.loc[(df['group']=='Heavy')&(df['direction']=='N')].count()[0]

38640

In [44]:
round(((df3.loc[df3['direction']=='N']['wheel_mass']*2).sum()/1000)/df.loc[df['group']=='Heavy'].count()[0],2)

20.67